# Hantush-Jacob 配线法

对

$$
s\cdot\frac{4\pi T}{Q}=W\left(u,\frac{r}{B}\right),\quad t\cdot\frac{4T}{r^2S}=\frac{1}{u}
$$

两边取对数

$$
\lg s + \lg\frac{4\pi T}{Q}= \lg W\left(u,\frac{r}{B}\right),\quad\lg t+\lg\frac{4T}{r^2S}=\lg\frac{1}{u}
$$

**思路：**

在标准曲线坐标系中移动 $s\sim t$ 散点图，根据平移量计算 $(T,S,\beta)$ ：

记坐标平移量为 

$$\Delta \lg(s)=\lg\frac{4\pi T}{Q},\quad \Delta \lg(t)=\lg\frac{4T}{r^2S}$$

$\beta$ 通过输入确定。

**参数计算公式：**

$$
T=\frac{Q}{4\pi}10^{\Delta\lg(s)},\quad S=\frac{4T}{r^2}10^{-\Delta\lg(t)}
$$

**例：作业6**

对某承压含水层抽水孔进行12h的定流量非稳定流抽水，抽水量为 $528m^3/d$，距抽水孔 90m  处有一观测孔，观测数据如下表 4，试确定水文地质参数（直线图解）。

<center> 表 4 承压含水层抽水试验观测数据

| 序号 | 时间(min) | 降深(cm) | 序号 | 时间(min) | 降深(cm) |
| :--: | :-------: | :------: | :--: | :-------: | :------: |
|  1   |     1     |   2.5    |  9   |    50     |   24.7   |
|  2   |     2     |   3.9    |  10  |    60     |   26.4   |
|  3   |     4     |   6.1    |  11  |    90     |   30.4   |
|  4   |     6     |   8.0    |  12  |    120    |   33.0   |
|  5   |     9     |   10.6   |  13  |    150    |   35.0   |
|  6   |    20     |   16.8   |  14  |    360    |   42.6   |
|  7   |    30     |   20.0   |  16  |    550    |   44.0   |
|  8   |    40     |   22.6   |  16  |    720    |   44.5   |

In [1]:
%matplotlib inline

import numpy as np
import matplotlib.pyplot as plt
import ipywidgets as widgets
import math as math
from wellfunction import *

# 控制小数的显示精度
np.set_printoptions(precision=4)

plt.rcParams['axes.unicode_minus']=False  #用来正常显示负号

In [2]:
# 准备数据, 根据配线法的种类，需要专门准备数据
Q = 528/1440  # m^3/min
r = 90 # m
t = np.array([1, 2, 4, 6, 9, 20, 30, 40, 50, 60, 90, 120, 150, 360, 550, 720]) # min
s = np.array([2.5, 3.9, 6.1, 8.0, 10.6, 16.8, 20.0, 22.6, 24.7, 26.4, 30.4, 33.0, 35.0, 42.6, 44.0, 44.5])/100 # m

ymin = 1.0e-2
ymax = 10.0

xmin = 1.0e-1
xmax = 1.0e4

x = np.linspace(-1, 4, 51)
x = np.float_power(10,x)

# 初始参数
T = Q/4/np.pi  # 配线法不同，公式也不一样
S = 4*T/r**2   # 配线法不同，公式也不一样
B = -999.0

In [3]:
# Plot the data
def Hantush_Jacob_fit(dlogs, dlogt, beta):
    global T, S, B # 设置全局变量，值可以穿透程序
    
    plt.style.use('default')
    fig, ax = plt.subplots()
    ax.set_xlim(xmin, xmax)
    ax.set_ylim(ymin, ymax)
    ax.set_xscale("log")
    ax.set_yscale("log")
    ax.set_aspect(1)
    plt.xlabel('$\log 1/u$')
    plt.ylabel('$W(u)$')
    ax.grid(True)
    
    T = Q/4/np.pi*np.float_power(10, dlogs)  # 配线法不同，公式也不一样
    S = 4*T/r**2*np.float_power(10, -dlogt)  # 配线法不同，公式也不一样
    u = r**2*S/4/T/x
    u0 = r**2*S/4/T/t
    ax.plot(x, hantush_jacob(u, beta),label="Hantush-Jacob曲线")
    ax.plot(x, theis(u),label="Theis曲线")
    ax.plot(t*np.float_power(10, dlogt), s*np.float_power(10, dlogs), '*', label="观测值")
    
    plt.legend(
        prop={'family': 'Simsun', 'size': 10}, handlelength=2,
        title="图例",
        title_fontproperties={'family': 'KaiTi', 'size': 12})
    ax.set_title("配线法", fontproperties={'family': 'KaiTi', 'size': 12}) # 指定显示中文字体

    plt.show()

    RSS = np.sum((hantush_jacob(u0, beta) - s)**2)

    print('             T(m^2/min): ', '{:.4f}'.format(T))
    print('                      S: ', '{:.4e}'.format(S))
    if beta>0 :
        print('                      B: ', '{:.4e}'.format(r/beta))
    print('Residual Sum of Squares: ', '{:.4e}'.format(RSS))

In [4]:
# ipywidgets 小部件控制参数, ipywidgets 有缓冲功能，同一个 Notebook 复制的代码得不到所期望的结果
widgets.interact(
    Hantush_Jacob_fit,
    dlogs = widgets.FloatSlider(
        value=0, min=-3.0, max=+3.0, step=0.01,
        description=r"$\Delta\lg s$ [-]:", continuous_update=False,
        readout_format='.2f', disabled=False),
    dlogt = widgets.FloatSlider(
        value=0, min=-5.0, max=+5.0, step=0.01,
        description=r'$\Delta\lg t$ [-]:', continuous_update=False,
        readout_format='.2f', disabled=False),
    beta = widgets.FloatSlider(
        value=0, min=0.0, max=+3.0, step=0.01,
        description=r'$\beta$ [-]:', continuous_update=False,
        readout_format='.2f', disabled=False),
    );   

interactive(children=(FloatSlider(value=0.0, continuous_update=False, description='$\\Delta\\lg s$ [-]:', max=…

**例：**

某河阶地，上部为潜水层，其下为 $2m$ 厚弱透水的亚砂土，再下为 $1.5m$ 厚的中、粗砂层(承压)。水源地抽取承压水，以 T32 号孔做非稳定抽水试验，距它 $197m$ 处有 T31 孔观恻，抽水量为 $Q=69.1m^3/h$，观测孔水位降深值如下表，求取水文地质参数。

<center> 表  某越流含水层抽水试验T31观测孔降深资料

| 抽水累计时间 $t(min)$ | 水位降深 $s(m)$ | 抽水累计时间 $t(min)$ | 水位降深 $s(m)$ | 抽水累计时间 $t(min)$ | 水位降深 $s(m)$ |
| :-------------------: | :-------------: | :-------------------: | :-------------: | :-------------------: | :-------------: |
|           1           |      0.05       |          60           |      0.575      |          300          |      0.763      |
|           4           |      0.054      |          75           |      0.62       |          330          |      0.77       |
|           7           |      0.12       |          90           |      0.64       |          360          |      0.77       |
|          10           |      0.175      |          120          |      0.685      |          390          |      0.785      |
|          15           |      0.26       |          150          |      0.725      |          420          |      0.79       |
|          20           |      0.33       |          180          |      0.735      |          450          |      0.792      |
|          25           |      0.383      |          210          |      0.755      |          480          |      0.794      |
|          30           |      0.425      |          240          |      0.76       |          510          |      0.795      |
|          45           |      0.52       |          270          |      0.76       |          540          |      0.796      |
